In [1]:
import sys
import time

import pickle as pkl
from matplotlib import pyplot as plt
import networkx as nx
import numpy as np


from Extractor import Extractor
from scipy.sparse import coo_matrix,csr_matrix
#import tensorflow as tf
import torch
from utils import *
from model import *
from train import *

from explain import *
import time
from tqdm import tqdm
from sklearn.metrics import roc_auc_score
from torch_geometric.utils import to_networkx


np.set_printoptions(threshold=sys.maxsize)

In [2]:
from torch_geometric.datasets import TUDataset, Entities
from torch_geometric.data import DataLoader
dataset = TUDataset(root='./dataset/Mutagenicity', name='Mutagenicity', use_node_attr=True)
dataset.len()

4337

In [3]:
train_len = int(0.8 * dataset.len())
test_len = dataset.len() - train_len

In [4]:
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_len, test_len])

In [7]:
#gcn_model = GCN(dataset, 2, 20)
#torch.load('./checkpoint/gcn_mutag')
gcn_model = ATTGCN(dataset, 2, 20)

In [6]:
gcn_model = torch.load('./MUTAG_search_checkpoint_2/auc_0.9326872275822641')
eval(gcn_model, test_dataset)

Epoch:  0 Avg loss:  0.47749466 ; acc:  0.7764977 ; epoch time:  0.4187736511230469


array(0.7764977, dtype=float32)

In [8]:
gcn_model.convs

ModuleList(
  (0): CD_GATConv(20, 10, heads=2)
)

In [10]:
train(gcn_model, train_dataset, test_dataset, max_epoch = 20, lr=0.001, save_model = True, temp_name = 'gcn_mutag_temp')

prepare dataloader
done
Epoch:  0 Avg loss:  0.536433 ; acc:  0.7287403 ; epoch time:  0.6443431377410889
eval test...
Epoch:  0 Avg loss:  0.5297241 ; acc:  0.74654377 ; epoch time:  0.07546472549438477
saving...


Epoch:  1 Avg loss:  0.5383674 ; acc:  0.73594695 ; epoch time:  0.6292674541473389
eval test...
Epoch:  0 Avg loss:  0.52801454 ; acc:  0.7453917 ; epoch time:  0.0765376091003418


Epoch:  2 Avg loss:  0.53855455 ; acc:  0.7296051 ; epoch time:  0.6795928478240967
eval test...
Epoch:  0 Avg loss:  0.5290939 ; acc:  0.735023 ; epoch time:  0.08679437637329102


Epoch:  3 Avg loss:  0.53617454 ; acc:  0.73392904 ; epoch time:  0.6770012378692627
eval test...
Epoch:  0 Avg loss:  0.5293412 ; acc:  0.75 ; epoch time:  0.08374619483947754
saving...


Epoch:  4 Avg loss:  0.5360531 ; acc:  0.73565865 ; epoch time:  0.6568953990936279
eval test...
Epoch:  0 Avg loss:  0.52571756 ; acc:  0.74769586 ; epoch time:  0.09054398536682129


Epoch:  5 Avg loss:  0.54359245 ; acc:  0.733

In [11]:
gcn_model = torch.load('./checkpoint/gcn_mutag_temp')
eval(gcn_model, test_dataset)

Epoch:  0 Avg loss:  0.52231944 ; acc:  0.75691247 ; epoch time:  0.09359312057495117


array(0.75691247, dtype=float32)

In [9]:
#torch.save(gcn_model, './checkpoint/gcn_mutag')

In [12]:
select = np.load('gt_index.npy')
edge_label_lists = np.load('edge_label_lists.npy', allow_pickle=True)

In [9]:
correct = 0
dataset.subgraph = True
dataset.remap = True
#load_model = torch.load('./checkpoint/gcn_mix').to('cuda')
load_model = gcn_model
load_model.eval()
in_correct = []
for idx in select:
    print('index: ', idx)
    data = get_data(dataset, idx).to('cuda')
    pred = load_model.forward(data)
    #print(preds)
    label = data.y.to('cuda')

    _, indices = torch.max(pred, 1)
    #print(pred)
    print(label)
    #print(torch.sum(label))
    print(indices)
    #print(torch.argmax(load_model(data)[0]))
    if (indices[0] == label[0]):
        correct += 1
    else:
        in_correct.append(idx)
print(correct)
print(in_correct)

index:  10
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
index:  22
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
index:  25
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
index:  31
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
index:  33
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
index:  41
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
index:  42
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
index:  44
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
index:  45
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
index:  49
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
index:  51
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
index:  65
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
index:  71
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
index:  72
tensor([0], device='cuda:0')
tensor([0], device='cuda:0')
index:  79
tensor([0], device='cud

In [13]:
kwargs = {}
kwargs['node_name_list'] = ['C', 'O', 'Cl', 'H', 'N', 'F', 'Br', 'S', 'P', 'I', 'Na', 'K', 'Li', 'Ca']

In [14]:
from utils import *
import torch
from CD_layers import *
import time
from torch_geometric.utils import to_networkx
import networkx as nx
import numpy as np
from matplotlib import pyplot as plt

def find_edge(model, dataset, idx, class_idx = None, node_sort = None, topk = None, start_num = 2):
    if node_sort is None:
        node_sort, node_color = print_explain(dataset, model, idx, class_idx = class_idx, visible = False)
    if class_idx is None:
        class_idx = int(dataset[idx].y[0])
    data = get_data(dataset, idx)
    data = data.to(device)
    start_num = start_num
    
    rest = list(node_sort.copy())
    select = set()
    neighbor_nodes = set()
    select_node_progress = []
    edge_index = np.array(dataset[idx].edge_index)

    while len(select) < topk and len(rest) != 0:
        if len(neighbor_nodes) == 0:
            for _ in range(start_num):
                if len(rest) == 0:
                    pass
                select_node = rest[0]
                select.add(select_node)
                rest.remove(select_node)
                # add new neighbor edge
                temp_neighbor_idx = (np.where(edge_index[0] == select_node))[0]
                #print(temp_neighbor_idx)
                for n_idx in temp_neighbor_idx:
                    n = int(edge_index[1][n_idx])
                    if n not in select:
                        neighbor_nodes.add(n)
                try:
                    neighbor_nodes.remove(select_node)
                except:
                    pass
                #print('no neighbor')
                #print('select: ', select)
                #print('neighbor_nodes', neighbor_nodes)
            select_node_progress.append(select.copy())
            
        else:
            # prepare list for score
            check_list = []
            for node in neighbor_nodes:
                group = select.copy()
                group.add((int)(node))
                check_list.append(group)
            #print(check_list)
            class_score = get_score(model, data, mask_list = check_list.copy())
            group_scores = class_score[class_idx]['rel']
            group_rank = np.argsort(group_scores)[::-1]
            print('score: ', group_scores[group_rank[0]])
            #print(group_rank)
            select = check_list[group_rank[0]]
            #print(check_list)
            intersect = select.intersection(neighbor_nodes)
            #print(intersect)
            neighbor_nodes = neighbor_nodes - select
            #print(rest)
            for node in intersect:
                rest.remove(node)
                # add new neighbor edge
                temp_neighbor_idx = (np.where(edge_index[0] == node))[0]
                #print('temp_neighbor_idx', temp_neighbor_idx)
                for n_idx in temp_neighbor_idx:        
                    n = int(edge_index[1][n_idx])
                    if n not in select:
                        neighbor_nodes.add(n)
            select_node_progress.append(select.copy())
        size = max(edge_index[0]) + 1
        adj = np.zeros((size, size))
        edge_pred = []
        for i in range(edge_index.shape[1]):
            r = edge_index[0][i]
            c = edge_index[1][i]
            if r in select and c in select:
                adj[r][c] = 1
                adj[c][r] = 1
            
            if r < c:
                if r in select and c in select:
                    edge_pred.append(1)
                else:
                    edge_pred.append(0)

        adj = preprocess_adj(adj)
    return select,select_node_progress,adj,edge_pred

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
def CD_explain(model, dataset, idx = 0, mask_node_list = None, node_range = None):
    """
    idx: idx of graph to explain
    """
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)

    data = get_data(dataset, idx)
    data = data.to(device)


    mask_list = []
    # generate mask
    node_range = data.x.shape[0] if node_range is None else node_range


    for i in range(node_range):
        if mask_node_list is not None and i in mask_node_list:
            mask_list.append(mask_node_list)
        else:
            mask_list.append([i])
    # forward to explain according to mask list
    class_score = get_score(model, data, mask_list)
    return class_score
def get_score(model, data, mask_list):

    for i in range(len(mask_list)):
        mask_index = [0] * data.x.shape[0]
        for m in mask_list[i]:
            mask_index[m] = 1
        mask_list[i] = mask_index
    
    preds_list = []
    for mask_index in mask_list:
        model.zero_grad()
        preds = model.forward(data, CD_explain=True, mask_index = mask_index)
        #print(preds)
        #print(data.y)
        preds['rel'] = preds['rel'].T
        preds['irrel'] = preds['irrel'].T
        #print(preds['irrel'].shape[0])
        #print(dataset.num_classes)
        preds = CD_softmax(preds, index = torch.tensor([0] * preds['irrel'].shape[0]).to(device))
        preds_list.append(preds)
        # need to softmax??

    # rel cd score for each class
    class_score = {}
    for class_idx in range(preds['irrel'].shape[0]):
        class_score[class_idx] = {}
        class_score[class_idx]['rel'] = []
        class_score[class_idx]['irrel'] = []
        for preds in preds_list:
            class_score[class_idx]['rel'].append((float)((preds['rel'][class_idx][0].cpu().detach())))
            class_score[class_idx]['irrel'].append((float)((preds['irrel'][class_idx][0].cpu().detach())))
            #class_score[class_idx]['rel'].append((float)((preds['rel'][0][class_idx].cpu().detach())))
            #class_score[class_idx]['irrel'].append((float)((preds['irrel'][0][class_idx].cpu().detach())))

    return class_score

def print_explain(dataset, model, idx, class_idx = None, visible = True, figsize = None, node_range = None, **kwargs):
    
    class_idx = class_idx if class_idx is not None else (int)(dataset[idx].y[0])
    
    begin_time = time.time()
    class_score = CD_explain(model, dataset, idx, node_range)
    
    print('prediction score',(class_score[0]['rel'][0] + class_score[0]['irrel'][0]))
    elapsed = time.time() - begin_time
    node_colors = class_score[class_idx]['rel']
    size_ratio = 1.0 + (node_colors - np.min(node_colors)) / (np.max(node_colors) - np.min(node_colors))
    print('epoch time: ', elapsed)

    if visible:
        if figsize is not None:
            plt.figure(figsize=figsize)
        G = to_networkx(dataset[idx])

        pos = nx.kamada_kawai_layout(G)

        node_sizes = 600 * size_ratio
        node_colors = class_score[class_idx]['rel']


        cmap = plt.cm.YlGnBu

        #t = [0, 1, 2, 4, 5, 6, 8, 9, 10, 17]
        #for tt in t:
        #    node_colors[tt] = 0.4264645576477051
        nodes = nx.draw_networkx_nodes(
            G,
            pos,
            node_size=node_sizes,
            node_color=node_colors,
            cmap=cmap,
            edgecolors='blue',
            vmax = max(node_colors) * 1.2,
            #vmin = 0.1
        )
        edges = nx.draw_networkx_edges(
            G,
            pos,
            node_size=node_sizes,
            arrowstyle="-",
            arrowsize=10,
            width=2,
        )
        label_list = {}
        for i in range(dataset[idx].x.shape[0]):
            label_list[i] = str(i)
            if 'node_name_list' in kwargs:
                node_name_idx = int(np.argmax(dataset[idx].x[i]))
                label_list[i] += '' + kwargs['node_name_list'][node_name_idx]

        labels = nx.draw_networkx_labels(G, pos, label_list, font_size=12, font_color="black")
        plt.colorbar(nodes)

        ax = plt.gca()
        ax.set_axis_off()
        #plt.savefig('1')
        plt.show()

    return np.argsort(node_colors)[::-1], node_colors

In [15]:
def Edge_explain(model, dataset, edge_list, idx = 0, node_range = None):
    """
    idx: idx of graph to explain
    """
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)

    data = get_data(dataset, idx)
    data = data.to(device)


    mask_list = []
    # generate mask
    node_range = data.x.shape[0] if node_range is None else node_range

    edge_index = data.edge_index
    
    mask_list = edge_list
    #for i in range(len(mask_list)):
    #    print(i, ' : ', mask_list[i])

    # forward to explain according to mask list
    class_score = get_score(model, data, mask_list)
    return class_score

In [16]:
edge_lists = np.load('edge_lists.npy', allow_pickle = True)

In [17]:
acc_list = []
auc_list = []
node_num_list = []

load_model = gcn_model
#load_model = torch.load('./checkpoint/gcn_mix').to('cuda')
load_model.eval()

all_node_label = []
all_node_color = []
#for idx in range(dataset.len()):
#for idx in range():
for idx in range(4337):
    #idx  = 21
    #if idx in in_correct:
    #    continue
    idx = int(select[idx])
    print('\nindex: ', idx)
    data = dataset[idx]
    #truth_node = np.where(sub_label[:,1] == True)[0]
    #truth_node = get_node_set(sub_edge_label_matrix)
    #if len(truth_node) > 6:
    #    continue
    print('label: ', data.y[0])
    node_sort, node_color = print_explain(dataset, load_model, idx, class_idx = 0, visible = False, figsize = (8,6), **kwargs)
    #_, _, adj, edge_pred = find_edge(load_model, dataset, idx, class_idx = None, node_sort = node_sort, topk = 10, start_num=1)
    #print(_)
    edge_score = Edge_explain(load_model, dataset, edge_lists[idx], idx)
    edge_colors = []
    for i in range(len(edge_score[0]['rel'])):
        edge_colors.append( edge_score[0]['rel'][i] - edge_score[1]['rel'][i] )
    print('edge sort: ', np.argsort(edge_colors)[::-1])
    auc = roc_auc_score(edge_label_lists[idx],edge_colors)
    print('auc: ', auc)
    auc_list.append(auc)
    print('auc mean: ', np.mean(auc_list))
    #print(roc_auc_score(edge_label_lists[idx][0:-1:2], edge_pred))
    #edge_type(adj, data)
    #node_label = np.array([0] * sub_label.shape[0]


index:  10
label:  tensor(0)
prediction score 0.8259669840335846
epoch time:  0.6753430366516113
edge sort:  [18 19 21 20 26 27 23 22  2  3 31 30 25 24 10 11 14 15 33 32 44 45 37 36
 50 51  9  8 29 28 47 46  1  0  5  4 16 17 49 48  6 41  7 40 34 35 39 38
 13 12 54 57 55 56 43 42 53 52]
auc:  0.9999999999999999
auc mean:  0.9999999999999999

index:  22
label:  tensor(0)
prediction score 0.9348153918981552
epoch time:  0.574183464050293
edge sort:  [26 27 29 28 33 32 18 19  4  5  0  1  8 22 23  9 20 21 10 11  3  2  6  7
 12 13 44 25 24 47 17 46 16 42 43 30 31 36 37 34 35 41 49 48 51 50 15 14
 40 45 39 38]
auc:  1.0
auc mean:  1.0

index:  25
label:  tensor(0)
prediction score 0.9078420102596283
epoch time:  0.7986199855804443
edge sort:  [18 19  7  6 63 62 61 60 16 17 30 31 34 35  1  0 29 28  8  9 26 27 54 57
 55 58 59 56 41 40  5  4 11 10 50 52 53 51 49 48  2  3 38 39 32 33 67 66
 65 12 13 64 69 68 25 24 22 15 14 23 36 37 44 43 47 46 45 42 20 21]
auc:  0.9393939393939393
auc mean:  0.9

KeyboardInterrupt: 

In [41]:
scores_2.append(get_score(load_model, data.to('cuda'), [[5,0,2,1,6,9,10,3,11]])[0]['rel'][0] - get_score(load_model, data.to('cuda'), [[5,0,2,1,6,9,10,3]])[0]['rel'][0])

In [42]:
scores_2.append(get_score(load_model, data.to('cuda'), [[5,0,2,1,6,9,10,3,13]])[0]['rel'][0] - get_score(load_model, data.to('cuda'), [[5,0,2,1,6,9,10,3]])[0]['rel'][0])

In [43]:
scores_2.append(get_score(load_model, data.to('cuda'), [[5,0,2,1,6,9,10,3,8]])[0]['rel'][0] - get_score(load_model, data.to('cuda'), [[5,0,2,1,6,9,10,3]])[0]['rel'][0])

In [44]:
scores_2.append(get_score(load_model, data.to('cuda'), [[5,0,2,1,6,9,10,3,4]])[0]['rel'][0] - get_score(load_model, data.to('cuda'), [[5,0,2,1,6,9,10,3]])[0]['rel'][0])

In [45]:
sort = np.argsort(np.absolute(scores_2))[::-1]
sort

array([0, 1, 4, 3, 2])

In [47]:
scores_2

[0.013296276330947876,
 -0.007134139537811279,
 -0.0032078325748443604,
 0.00396081805229187,
 0.006972581148147583]